In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import seaborn as sns
from sklearn.cluster import KMeans
import plotly.express as px
import plotly.subplots as sp
from sklearn.decomposition import PCA
from sklearn.metrics import (
    calinski_harabasz_score,
    davies_bouldin_score,
    silhouette_score,
)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from umap.umap_ import UMAP
from yellowbrick.cluster import kelbow_visualizer, silhouette_visualizer

from utils import (
    create_interactive_pie_charts,
    plot_feature_correlations,
    plot_feature_distributions,
    detection_outlier
)

In [ ]:
# Load the data
data = pd.read_csv("customer_data_test.csv", sep=";", index_col=0)
data = data.drop(columns=["ClientId"])

In [ ]:
# Display the first 5 rows of the data
data.head()

In [ ]:
data.info()

In [ ]:
# Check for missing values
data.isnull().sum()

In [ ]:
# Check for duplicates
data.duplicated().sum()

In [ ]:
# Check for unique values
data.nunique()

In [ ]:
# Check negative values
num_negative_features_per_col = (data < 0).sum()
num_negative_features_per_col

In [ ]:
# Drop columns with negative values in columns TotalInactiveDays and ActivePassiveRatio
columns_to_check = ['TotalInactiveDays', 'ActivePassiveRatio']
negative_indices = data[columns_to_check].map(lambda x: x < 0).any(axis=1).index[data[columns_to_check].map(lambda x: x < 0).any(axis=1)]
data = data.drop(negative_indices)

In [ ]:
# Describe the data
percentiles = [0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99]
data.describe(percentiles=percentiles)

In [ ]:
# Get the most correlated features without duplicates
corr_matrix = data.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
most_correlated = upper.stack().sort_values(ascending=False)
most_correlated = most_correlated[most_correlated > 0.5]
most_correlated

### Feature analysis

In [ ]:
data.describe(percentiles=percentiles)

In [ ]:
plot_feature_correlations(data)

In [ ]:
plot_feature_distributions(data)

In [ ]:
# Binned features analysis
create_interactive_pie_charts(data, num_bins=10)

### Data insights
*Note: features that described days are changed to have a range between 0 and 365*

- 29% of users won more than they lost
- 82.3% of users have up to 34 active days per year
- 47.1% of users have up to 37 days with deposits per year (it means that 47.4% of users have money only on activity days)
- 72.4% of users have up to 587 days before their first deposit
- 95.7% of users have up to 585 bets per year
- 93.7% of users have up to 4 sport type of bets  
- 97.1% of users have an average sports bet of up to 116 EUR

### Clustering

In [ ]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data)
pca = PCA(n_components=scaled_data.shape[1])
pca.fit(scaled_data)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel("Number of components")
plt.ylabel("Cumulative explained variance")
plt.grid()
plt.show()

In [ ]:
n_components_list = [0.8, 0.9]
n_clusters_list = list()
silhouette, calinski, davies = list(), list(), list()
for n_components in n_components_list:
    pca = PCA(n_components=n_components)
    X = pca.fit_transform(scaled_data)
    print(
        f"Number of components for {n_components} explained variance: {pca.n_components_}"
    )
    kmeans = KMeans(n_init=30, max_iter=1000)
    visualizer_1 = kelbow_visualizer(kmeans, X, k=(2, 12))
    kmeans = KMeans(n_clusters=visualizer_1.elbow_value_, n_init=30, max_iter=1000)
    visualizer_2 = silhouette_visualizer(kmeans, X)
    labels = kmeans.fit_predict(X)
    n_clusters_list.append(visualizer_1.elbow_value_)
    print(f"Silhouette score: {silhouette_score(X, labels)}")
    print(f"Calinski-Harabasz score: {calinski_harabasz_score(X, labels)}")
    print(f"Davies-Bouldin score: {davies_bouldin_score(X, labels)}\n")

n_clusters_list = np.unique(n_clusters_list).tolist()

extended_n_cluster_list = []
for n in n_clusters_list:
    if n - 1 >= 2:  # Check to ensure no negative values
        extended_n_cluster_list.append(n - 1)
    extended_n_cluster_list.append(n)
    extended_n_cluster_list.append(n + 1)

# Remove duplicate values and sort the list
n_clusters_list = sorted(list(set(extended_n_cluster_list)))

In [ ]:
# Define the objective function for Optuna


def objective(trial):
    # Define hyperparameters to tune
    n_components = trial.suggest_categorical("pca__n_components", n_components_list)
    n_clusters = trial.suggest_categorical("kmeans__n_clusters", n_clusters_list)
    n_init = trial.suggest_categorical("kmeans__n_init", [10, 20, 30])
    max_iter = trial.suggest_categorical("kmeans__max_iter", [300, 500, 1000])

    # Preprocessing pipeline
    preprocessing = Pipeline(
        [("scaler", StandardScaler()), ("pca", PCA(n_components=n_components))]
    )

    # Create pipeline
    pipe = Pipeline(
        [
            ("preprocessing", preprocessing),
            (
                "clusterer",
                KMeans(n_clusters=n_clusters, n_init=n_init, max_iter=max_iter),
            ),
        ]
    )

    pipe.fit(data)

    return silhouette_score(
        pipe.named_steps["preprocessing"].transform(data),
        pipe.named_steps["clusterer"].labels_,
    )


search_space = {
    "pca__n_components": n_components_list,
    "kmeans__n_clusters": n_clusters_list,
    "kmeans__n_init": [10, 20, 30],
    "kmeans__max_iter": [300, 500, 1000],
}

# Create a study for grid search
study = optuna.create_study(
    direction="maximize", sampler=optuna.samplers.GridSampler(search_space)
)
n_experiments = np.prod([len(v) for v in search_space.values()])
print(f"Number of experiments: {n_experiments}")
study.optimize(objective, n_trials=n_experiments)
# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=10)

# Print the best parameters
print("Best parameters found: ", study.best_params)
print("Best Silhouette Score: ", study.best_value)

In [ ]:
pca = PCA(n_components=study.best_params["pca__n_components"])
X = pca.fit_transform(scaled_data)

kmeans = KMeans(
    n_clusters=study.best_params["kmeans__n_clusters"],
    n_init=study.best_params["kmeans__n_init"],
    max_iter=study.best_params["kmeans__max_iter"],
)

In [ ]:
visualizer_2 = silhouette_visualizer(kmeans, X)

In [ ]:
# Example creation of DataFrame for demonstration
pca_components = pd.DataFrame(X)

# Rename PCA components to PCA_1, PCA_2, etc.
pca_components.columns = [f"PCA_{i+1}" for i in range(pca_components.shape[1])]

# Initialize an empty DataFrame to store correlations
correlation_matrix = pd.DataFrame(index=data.columns, columns=pca_components.columns)

# Calculate correlations between each feature and each component
for feature_column in data.columns:
    for pca_column in pca_components.columns:
        correlation_matrix.loc[feature_column, pca_column] = np.corrcoef(
            data[feature_column], pca_components[pca_column]
        )[0, 1]

# Convert data to numeric format
correlation_matrix = correlation_matrix.astype(float)

plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Correlation Matrix of Original Features with PCA Components")
plt.show()

In [ ]:
# Preprocessing pipeline
preprocessing = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("pca", PCA(n_components=study.best_params["pca__n_components"])),
    ]
)

# Create pipeline
pipe = Pipeline(
    [
        ("preprocessing", preprocessing),
        (
            "clusterer",
            KMeans(
                n_clusters=study.best_params["kmeans__n_clusters"],
                n_init=study.best_params["kmeans__n_init"],
                max_iter=study.best_params["kmeans__max_iter"],
            ),
        ),
    ]
)

In [ ]:
labels = pipe.fit_predict(data)
data["label"] = labels

### Clusters analysis

In [ ]:
data.groupby("label").describe(percentiles=percentiles).T

In [ ]:
# Split features and cluster labels
features = data.columns[:-1]  # Assuming the 22nd feature is the cluster number
cluster_col = data.columns[-1]

# Number of features
num_features = len(features)

# Define the number of rows and columns for subplots
num_cols = 4
num_rows = (num_features + num_cols - 1) // num_cols

# Create subplots
fig = sp.make_subplots(rows=num_rows, cols=num_cols, subplot_titles=features)

# Generate box plots for each feature
for i, feature in enumerate(features):
    row = i // num_cols + 1
    col = i % num_cols + 1
    fig.add_trace(
        px.box(data, x=cluster_col, y=feature).data[0],
        row=row, col=col
    )

# Update layout
fig.update_layout(
    height=num_rows * 400,
    width=1000,
    title_text="Box-Plots for Each Feature by Cluster",
    showlegend=False
)

fig.show()

### Clusters visualization

In [ ]:
# Define the objective function for Optuna


def objective(trial):
    # Define hyperparameters to tune
    n_components = 2
    n_neighbors = trial.suggest_int("n_neighbors", 10, 200, step=10)
    min_dist = trial.suggest_categorical(
        "min_dist", np.arange(0.1, 1, 0.1).tolist() + [0.99]
    )

    reducer = UMAP(
        n_components=n_components, n_neighbors=n_neighbors, min_dist=min_dist
    )
    embeddings = reducer.fit_transform(X)

    return silhouette_score(embeddings, labels)


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)

# Print the best parameters
print("Best parameters found: ", study.best_params)
print("Best Silhouette Score: ", study.best_value)

In [ ]:
reducer = UMAP(
    n_components=2,
    n_neighbors=study.best_params["n_neighbors"],
    min_dist=study.best_params["min_dist"],
)
embeddings = reducer.fit_transform(X)

In [ ]:
scatter = plt.scatter(
    embeddings[:, 0], embeddings[:, 1], c=labels, s=1, cmap="Spectral"
)
legend1 = plt.legend(*scatter.legend_elements(), title="Labels")
plt.gca().add_artist(legend1)
plt.xlabel("Embedding Dimension 1")
plt.ylabel("Embedding Dimension 2")
plt.title("Scatter Plot of Clusters")
plt.show()